# RNA-seq data analysis

**First Summer School on Bioinformatics**  
June - 2021  
Tampere University  
Author: Ebrahim Afyounian  

In this exercise, we are going to work with **gene expression quantification** from **RNA-seq** experiment from [this](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70466) study.

- &#x1F4DD; **Activity**: Follow the link above to familiarize yourself with the type of samples in the data set.
    - What are the two cell lines from which this data set is composed of?
    - Which cell line in this experiment represents the normal cells?
    - Which cell line in this experiment represents the tumor cells?
    - How many samples are there from each cell line? 
    - Which sequencing platform was used for sequencing?

&#x26A0; **Note**: In what follows, the word "cell" refers to a box in this notebook that can be run by pressing "<kbd>ctrl</kbd>+<kbd>Enter</kbd>" buttons. If we want to refer to a biological cell, we will make it clear.

In [ ]:
## This is a cell; run it by pressing "ctrl+Enter"
1+1

## Data preparation

Here, we jump over the details about how the abundances of each transcript in each sample has been determined for this experiment. If you are interested however, follow [this](https://pachterlab.github.io/kallisto/about) link.

The result of quantification is available in the following file:
`https://raw.githubusercontent.com/eafyounian/2021_summer_school_bioinf/main/resources/GSE70466-expression.txt`

To be able to analyze the data using R programming language, we need to load it in R environment.

In R, we can load a file with different functions such as `read.table()` or `read.csv()`. 

To learn how to use a particular function in R, we can read the help/manual page for that function. For example, to read the manual for `read.table()` function, we can use `help(read.table)` or `?read.table`.
- &#x1F4DD; **Activity**: Try running `?read.table` in the following cell and spend a few moment to familiarize yourself with the function.

In [ ]:
?read.table

Running the following cell, load the quantification into a data frame called `df`.


In [ ]:
df = read.table('https://raw.githubusercontent.com/eafyounian/2021_summer_school_bioinf/main/resources/GSE70466-expression.txt',
                sep='\t', 
                header=1, 
                stringsAsFactors = FALSE,
                row.names=1,
               )

We can use the `head()` function, like `head(df)` to inspect the first few rows in the data frame. 
- &#x1F4DD; **Activity**: Based on the output of the following cell, tell what the columns and the rows are (i.e. whether they are genes or samples)?

In [ ]:
head(df)

To see how many rows and how many column are there in our data frame `df`, we can use the `dim()` function that returns the _**dim**entions_ of the data frame.
- &#x1F4DD; **Activity**: Based on the output of the following cell, how many genes have been tried to be quantified?

In [ ]:
## Do the activity in this cell

Even though it seems that there are `35238` rows in the data, not all of them will be informative because there are some rows that have similar values across all the columns. This could be a row with all `0`s or another number. We can discard these rows (this is an instance of **data preparation**). 

Let's see how many of such rows do exist in our data frame. One way to find out is to find the variance across  all the columns for each row and check the results. If the result for a row is `0`, we know that all columns  for that row have equal values (for example try this and check the result: `var(c(1, 1, 1))`).


To calculate the variance over the columns for each row, we could write a `for` loop that calculates the variance one rows at a time, but we can also use R `apply()` function `apply(df, 1, var)` for this purpose. This function, applies the `var()` function to each as row denoted by `1` as the second function argument (see `?apply` for more info). 


Let's run the following cell and look at the first `20` rows. We can see that there are `2` rows with variance equal to `0`.

In [ ]:
df_vars = apply(df, 1, var)
head(df_vars, 20)

Now, let's remove the rows that have variance `0`. The other way to formulate this task is to **keep** the rows that do **not** have variance `0`. We can achieve this by running the following cell.

Let's see what is happening in the following cell: `df_vars != 0` results in a Boolean vector where the value for a particular row is equal to `TRUE` if the the var is **non-zero** and `FALSE` otherwise. Next, we use this vector to create a subset of our data frame `df` and assign the results to a variable called `df_nonzero_var`. The `df[df_vars != 0, ]` tells R to keep only rows the are `TRUE` after the `df_vars != 0` expression is evaluated and also to keep all the columns.

- &#x1F4DD; **Activity**: After running the following cell, check the dimensions of the new variable i.e. `df_nonzero_var` and tell how many of the rows were **filtered out**.

In [ ]:
df_nonzero_var = df[df_vars != 0,]

In [ ]:
## Do the activity in this cell

So far, we have loaded the data and performed some data preparation on it. Now, let's do some analysis. Each analysis may require more data preparation but we discuss them when needed.

## Principal component analysis (PCA)

Often, we are interested to visually inspect our data and check whether we can find any noticeable pattern in it. For example, we might be interested to see how similar or different they are from each other. 

When doing visualizations, we can go up to 3 dimensions. However, we have noticed that our data has thousands of rows and several columns (i.e. our data is high-dimensional). Thus we need a way to reduce the dimensions in our data set to be able to visualize it and yet this reduction in the dimensions should preserve the information in the data set as much as possible. 

One way to achieve this goal is to use a statistical method called **Principal Component Analysis** (PCA). In our particular application, this method takes gene expression values and transforms them to what is called as **principal components** (PCs). These are a set of linearly uncorrelated features that capture the sources of variance in the data.

Once we have the PCs, we can use the first two or three principal components and visualize them in either a 2-dimensional (2-D) or 3-dimensional (3-D) scatter plot respectively.

After visualization, if two data points (i.e. two RNA-seq samples) are close together, it means that they are highly correlated or in other words they are similar in their gene expression profiles.

One thing that we need to make sure before PCA analysis, is to check whether our data is **centered** around `0` and whether our data is **scaled** to unit variance (i.e. `1`) because to work properly, PCA requires these conditions to be satisfied. If not, we need to tell the R function that does the PCA analysis for us to center and scale the data.

&#x1F4DD; **Activity**: Use `rowMeans()` function to see whether the data is centered. Also inspect the `df_vars` to check whether the data is scaled to unit variance.

In [ ]:
## Do the activity in this cell

Now, let's perform PCA analysis on our data. We can use  `prcomp()` function to do so (run `?prcomp` to familiarize yourself with the function). 

This function accepts two arguments `scale` and `center` that we can set based on the result of the activity above. They tell the function whether we want to scale or center the data.

**Note**: `prcomp()` function expects the rows to be samples and columns to be genes. But our data is in the opposite order. Thus we need to transpose our data. To do so, we are using the R `t()` function.

&#x1F4DD; **Activity**: Set the values for these two arguments before running the cell below i.e. replace ... with either `TRUE` or `FALSE`.

In [ ]:
pca_results = prcomp(t(df_nonzero_var), scale=..., center = ...)

We can use R `summary()` function to get a summary of the PCA result.
Let's inspect the result by running the following cell.

&#x1F4DD; **Activity**: 
- How many PCs are resulted from the PCA analysis?
- Is there a relation between the number of PCs and the number of samples?
- What percentage of the variance in the data is explained by the first two PCs? 

In [ ]:
summary(pca_results)

We can plot a 2-D plot using the first `2` PCs using the following cell. For visualization, we are using `ggplot` package. In this exercise, we skip the visualization details.

&#x1F4DD; **Activity**:
- Are cancer samples separated from normal samples?
- If yes, along which PC, this separation is visible?
- Are normal samples, more similar to each other or cancer samples?

In [ ]:
library(ggplot2)


## Creating a data frame out of PCA analysis result
pca_results_df = data.frame(Sample = rownames(pca_results$x),
                             X = pca_results$x[,1],
                             Y = pca_results$x[,2])

## Calculating the variance explained by each principal components
pca_var = pca_results$sdev^2
pca_var_percent = round(pca_var/sum(pca_var)*100, 1)

## Plotting
ggplot(data = pca_results_df, aes(x = X, y = Y, label = Sample)) +
geom_point() + 
scale_x_continuous(limits = c(-125, 125)) + 
scale_y_continuous(limits = c(-125, 125)) +
geom_text(mapping = aes(x = X, y = Y+10, label = Sample), size = 5) +
  xlab(paste("PC1: ", pca_var_percent[1], "%")) + 
  ylab(paste("PC2: ", pca_var_percent[2], "%")) + 
  ggtitle("PCA analysis result") +
  theme_bw()

Another way to inspect, analyze our high-dimensional data, is via clustering the genes and/or samples. For example, clustering genes groups together genes with similar expression profile across samples. There are 3 major classes of clustering methods: 1) hierarchical clustering, 2) partitioning, and 3) density-based methods. Here, we apply hierarchical clustering to our data. Often, the hierarchical results is followed by visualization using heat maps as we will see below.

## Hierarchical clustering

To perform hierarchical clustering, we need to have a measure that tells the distance between two samples/genes (the smaller the distance, the more similar the samples/genes are; this is referred to as *distance metric*). Once we have a way to calculate the distance, we start by grouping the samples/genes with the least distance to each other and continue until all samples/genes are grouped. In this approach, sometimes we need to find the distance of a sample/gene to a groups of samples/genes. There are few ways to find such distances (this is referred to as *linkage method*). For example, to calculate the distance between a sample *A* and a group/cluster of 2 samples (*B* and *C*) using the "complete" linkage method, we use the maximum distance of *A* and *B* or *A* and *C* to be the distance between *A* and the cluster that contain *B* and *C*.

There are several distance metrics available e.g. Euclidean distance, Manhattan distance, and correlation distance to name a few. The choice of the proper distance metric is important. For example, it has been known that Euclidean distance is sensitive to scaling and differences in average expression level whereas correlation is more robust to these changes. In this example, we use **correlation distance** for **distance metric** and **complete** linkage for **linkage method**.

We use R `hclust()` function to perform hierarchical clustering (use `?hclust` to familiarize yourself with the function). using the R `plot()` function, we can visualize the result which is a *cluster dendrogram* which shows the clusters on the X-axis as well as the distance between the clusters on the Y-axis.

In [ ]:
## defining a function that calculates the correlation distance
correlation_dist = function(x, method = "pearson"){
    corr_distance = as.dist((1-cor(x, method=method))/2) 
    return(corr_distance)
}

## Hierarchical clustering
hc = hclust(correlation_dist(df_nonzero_var), method = "complete")

## plotting the hierarchical clustering results
plot(hc)

&#x1F4DD; **Activity**:
- Look at the Y-axis of the plot above! What is the approximate distance between `GSM1778906` and `GSM1778907`? 
- Look at the Y-axis of the plot above! What is the approximate level of Pearson correlation between `GSM1778906` and `GSM1778907`?
- Compare PCA plot with the hierarchical clustering plot. Are the two closest normal samples in the PCA differ from the two closest normal samples in the hierarchical clustering? What could be the reason?
- In the following cell, change the distance metric to Euclidean distance (**hints**: you need to use the R `dist()` function. Also, you need to transpose the data). 
    - Do you see any difference between this and earlier plot? Would scaling the data change the final result? 

In [ ]:
## Do the activity in this cell

### Heat map

As mentioned above, we can use heat map to visualize our data after the hierarchical clustering. Here, we use `pheatmap` R package which provides us with `pheatmap()` function to perform both hierarchical clustering as well as visualization.

Below, we are using the top 1000 most variable genes for the visualization.

&#x1F4DD; **Activity**:
- Name a couple of reasons why we only use a fraction of data for this purpose?
- Compare the clustering of the samples with earlier results. Do you see any difference?
- Look at the heat map, are there groups of genes that behave differently across different sample types (i.e. normal and tumor)?
- What is the purpose of setting `scale = 'row'` (**hint**: ?pheatmap)?
- What is the purpose of setting `show_rownames=FALSE` (**hint**: ?pheatmap)? 

In [ ]:
library(pheatmap)

df_vars_order = order(df_vars, decreasing = TRUE) # returns the indices of rows when order in a decreasing order
df_ordered = df[df_vars_order,] # Using the indices from above to reorder our data frame; Remember that df 
                                # contain also rows with variance 0, but after reordering, these row are all
                                # at the bottom.

pheatmap(as.matrix(df_ordered[1:1000,]), 
         scale = 'row',
         clustering_distance_rows = 'correlation', # Tells to use the Pearson correlation distance for clustering
         clustering_distance_cols = 'correlation',
         show_rownames = FALSE,  
         main = 'Heat map of top variable genes',
        )

## Differential expression analysis using `DESeq2`

Based on the heat map, we see that there genes that are more expressed in normal samples than in tumor sample or vice versa. One might hypothesize that these genes have something to do with the development of cancer or the absence of it. So, how can we reliably find such genes that are differentially expressed (DE) across different sample types? 

At its simplest, we can go through genes, one gene at a time, and use an appropriate statistical test that tells whether the two groups are different from each other in terms of their gene expression (our null hypothesis is that the groups are similar or data from both groups come from one distribution). However, we have to take into consideration several possible issues. 

For example, to be able to compare the groups, they need to be comparable. Thus, we need a way to normalize the data before the comparison. But what is a proper normalization approach for our data? 

For another example, consider answering the question of what is the appropriate statistical test for our purpose knowing that each statistical test makes assumptions about the underlying data distribution. What is the distribution of our data? 

Let's consider another example: as we have thousands of genes to test, we end up in a multiple hypothesis testing situation. If we do not correct for this, in our analysis result, we might have some genes labeled as DE where in reality they are not (**note**: by removing the genes with variance `0`, we somehow alleviate this problem because in total, we do less test but the problem still exists). 

Luckily, there has been several statistical methods developed for finding DE genes that try to address many of the issues relevant to the differential expression analysis (e.g. [DESeq2](http://bioconductor.org/packages/release/bioc/html/DESeq2.html), [limma](http://bioconductor.org/packages/release/bioc/html/limma.html), and [edgeR](http://bioconductor.org/packages/release/bioc/html/edgeR.html) ). 

Here, we use a popular method called `DESeq2`. `DESeq2` starts by using the count data and performs all required normalization, modeling, testing, and multiple hypothesis testing correction for us.  

In [ ]:
## loading DESeq2
library(DESeq2)

## Loading the meta data for our samples
coldata = read.csv('https://raw.githubusercontent.com/eafyounian/2021_summer_school_bioinf/main/resources/GSE70466-metadata.txt', 
                    sep = '\t', 
                    row.names = 'Sample_geo_accession',
                   )
coldata = coldata['cell.line']

## creating DESeqDataSet
dds = DESeqDataSetFromMatrix(countData = as.matrix(df_nonzero_var), 
                              colData = coldata, 
                              design = ~cell.line
                             )

## Setting the cell line against which the comparison is performed 
dds$cell.line <- relevel(dds$cell.line, ref = "PrEC")

## Perform the differential expression analysis
dds = DESeq(dds)

## Correcting for the multiple hypothesis issue
res = results(dds, alpha = 0.0001)

## Keeping only genes that their adjusted p-value is less than 0.0001.
## This threshold sets the FDR to 0.01% i.e. 0.01% of our DE genes are false positives
res_subset = subset(res, padj < 0.0001)

## Ordering the DE genes based on the log2 fold change
res_subset_ordered = res_subset[order(res_subset$log2FoldChange),]

## Write the results into file(s) for future use
write.table(as.data.frame(res_subset_ordered), 
            file = 'LNCaP_vs_PrEC_DE_genes.txt', 
            sep = '\t', 
            quote=FALSE
           )

write.table(rownames(res_subset_ordered), 
            file = 'LNCaP_vs_PrEC_DE_gene_names.txt', 
            sep = '\t', 
            quote=FALSE, 
            row.names=FALSE
           )


## Looking at the summary of the data
summary(res_subset_ordered)

&#x1F4DD; **Activity**:
- Use the `head(res_subset_ordered, 1)` to see the first DE gene. 
    - What is the name of this gene?
    - In which sample type is this gene more expressed (up-regulated)?
- Use the `tail(res_subset_ordered, 1)` to see the last DE gene. 
    - What is the name of this gene?
    - In which sample type is this gene more expressed?
- Inspect the summary results in the above cell.
    - How many genes are DE?
        - OF these DE, around how many do we expect to be false positive?
    - How many genes are up-regulated in LNCaP?
    - How many genes are down-regulated in LNCaP?

In [ ]:
## Do the activity in this cell

In [ ]:
## Do the activity in this cell

Let's plot the heat map of only the DE genes similar to the heat map we created above.

&#x1F4DD; **Activity**:
- Compare this heat map with the earlier heat map. Name some of the changes that you notice?

In [ ]:
## Subsetting our original data frame by keeping only DE genes
DE_gene_expr = df[rownames(res_subset),]

pheatmap(as.matrix(DE_gene_expr), 
         scale = 'row',
         clustering_distance_rows = 'correlation', # Tells to use the Pearson correlation distance for clustering
         clustering_distance_cols = 'correlation',
         show_rownames = FALSE,  
         main = 'Heat map differentially expressed genes',
        )

**Note**: Here, we just scratched the surface using `DESeq2`. For more more information, please read [this](http://www.bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html) tutorial written by the creators of the tool.

So far so good! We now have a list of genes that are differentially expressed across the two sample types. How are we going to make sense of this list? One of the simplest ways is to go through the list, one gene at a time, and see if we can spot a gene that sounds interesting with respect to the disease we are studying. But we have thousands of DE genes in this experiment. One way to approach this issue here is via enrichment analysis. The basic idea is to see whether e.g. a biological pathway is enriched with DE genes. If we find such a pathway, we could hypothesize that this pathway may be related to the condition under study. 

##  Functional and enrichment analysis

Here, we perform functional and enrichment analysis over all differentially expressed genes using an [online service](https://david.ncifcrf.gov/home.jsp) called Database for Annotation, Visualization and Integrated Discovery (DAVID).

When we performed differential expression analysis, we saved the results in a file called: `LNCaP_vs_PrEC_DE_gene_names.txt`. Following the steps below, we will check if we can find any enriched pathways.

- Go to the GenePattern Notebook Home Page tab in your browser. You should be able to see a file named: `LNCaP_vs_PrEC_DE_gene_names.txt`. Download this device to your local machine by selecting the check box next to the file and pressing the <kbd>Download</kbd> button that appears. We will upload this file into DAVID. 
- Open a new browser tab by following [this](https://david.ncifcrf.gov/home.jsp) link.
- On the web-page, click on the Start Analysis. 
- On the left panel make sure that upload tab is selected. 
    - In Step 1: `Enter Gene List`, select <kbd>Choose File</kbd> and upload the file you downloaded to your local machine. 
    - In Step 2: `Select Identifier` select: `OFFICIAL_GENE_SYMBOL`.
    - In Step 2a: `Select species` select `Homo sapiens` by typing it in the box.
    - In Step 3: `List Type`, select `Gene List`. 
    - In Step 4, press the <kbd>Submit List</kbd> button.
- Now, you should be able to see the message: `Step 1. Successfully submitted gene list`.
- Under `Step 2. Analyze above gene list with one of DAVID tools`, press the `Functional Annotation Tool` link.
- Under `Annotation Summary Results`, select `Pathways`.
- Finally, press the <kbd>Chart</kbd> button next to `KEGG_PATHWAY`. Inspect the results in the window that opens up. The results are already sorted based on the Benjamini-adjusted p-values.

&#x1F4DD; **Activity**:
- Do you spot any interesting enriched pathways?
- Under the term column, press the link to one enriched pathway that interests you. This should open up a pathway chart with DE genes marked with blinking starts on the pathway. 

## Conclusion

Here, we conclude this very short analysis of RNA-seq data. Here, we scratched only the surface. We did not go very deep into the details of the analyses we did, and we left out many other possible analyses that could have been done. 

### Further reading

Here are some resources that might be helpful for people interested in further learning about the RNA-seq analysis.
- [A survey of best practices for RNA-seq data analysis](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0881-8)
- [RNA-seqlopedia](https://rnaseq.uoregon.edu/)
- [DESeq2 tutorial](http://www.bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html)
- [Informatics for RNA-seq: A web resource for analysis on the cloud](https://github.com/griffithlab/rnaseq_tutorial)
- [Pathway Commons guid on RNA Sequencing Analysis](https://www.pathwaycommons.org/guide/primers/data_analysis/rna_sequencing_analysis/)

Thank you for your attention and good luck! &#x1F389;